## <font color = yellow> A single Sale will combine the first row from the Buys & Sells dfs

In [1]:
library(tibble)

In [2]:
cat("One", "Two", "Three", sep = " | ", fill = TRUE)

One | Two | Three


In [3]:
options(pillar.width = Inf)


## <font color = yellow> create_df function () - reads .csv

In [4]:
   
df <- read.csv("fifo_new.csv", header = TRUE, sep = ",")
df$date <- as.Date(df$date, origin = "1899-12-30")

In [5]:
head(df, 2)


,index,date,type,co,quan,pps,basis,sale_quan,tally,cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,Sell_cf,Sell_cf_pps,Sell_cf_basis,TorF
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<int>,<int>,⋯,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<int>
1,1,2022-12-31,Buy,AMC,1050,4.0700,4273.5,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
2,2,2023-01-27,Sell,AMC,1000,5.5124,5512.4,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


## <font color = yellow> Create dfs

In [6]:
#    df_records <- data.frame()
sale_records <- data.frame() 


In [7]:
# create dfs from filtered df
buys <- df[df$type == 'Buy', ]  # Filter buys
sells <- df[df$type == 'Sell', ]  # Filter sells
b <- buys[1,]
s <- sells[1,]

## <font color = yellow> Basic updates when a sale occurs


            sale <- s$quan
            bs_ratio <- b$quan / s$quan
            cf_quan <-  b$quan - s$quan

            # assign the cf values - if they exist
            if(b$quan > s$quan) {
                b$cf_quan <- (b$quan - s$quan)        
                b$cf_pps <- b$pps
                b$cf_basis <- cf_quan * b$pps
            }
            b$sale_quan <- b$quan
            b$buy_quan <- s$quan 
            b$buy_pps <-  b$pps
            b$buy_basis <-  b$buy_quan * b$pps

            b$sell_quan <- s$quan
            b$sell_pps <- s$pps
            b$sell_basis <- s$quan * s$pps

            b$profit_loss <- b$sell_basis - b$buy_basis
            b$sale <- sale

            s$type <- "SOLD"

            return(b)   



In [8]:
nrow(buys)
nrow(sells)
nrow(sale_records)

[1] 23

[1] 18

[1] 0

## <font color = yellow> Update b

In [9]:
for(i in 1:3) {
    cat("for loop, loop is i: ", i, "\n")
    }

for loop, loop is i:  1 
for loop, loop is i:  2 
for loop, loop is i:  3 


In [10]:
# update b based on Sale Type: 

update_b <- function() {

        sale <- s$quan
        bs_ratio <- b$quan / s$quan
        cf_quan <-  b$quan - s$quan

        # assign the cf values - if they exist
        #  b > s
        if ( b$quan > s$quan) {    
            
            # b$sale_quan <- b$quan
            b$buy_quan <- s$quan
            b$buy_pps <-  b$pps
            b$buy_basis <-  b$buy_quan * b$pps


            b$sell_quan <- s$quan    # cannot sell more than the buy_quan
           # b$sell_quan <- b$quan
            b$sell_pps <- s$pps
            b$sell_basis <- s$quan * s$pps

            b$cf_quan <- (b$quan - s$quan)        
            b$cf_pps <- b$pps
            b$cf_basis <- cf_quan * b$pps           

            b$profit_loss <- b$sell_basis - b$buy_basis
            # b$sale <- sale

            s$type <- "Buy-CF"
    
       # ( s > b ), has sell carryforward
 # cat("If s > b, then S_cf_quan is: ", S_cf_quan, " and Sell_cf is ", b$Sell_cf, "\n")  
        }else if(s$quan > b$quan) {
            
            
            b$sale_quan <- b$quan
           # b$buy_quan <- s$quan 
            b$buy_quan <- b$quan
            b$buy_pps <-  b$pps
            b$buy_basis <-  b$buy_quan * b$pps

        
            # Sell c/f
            S_cf_quan <- s$quan - b$quan           
            b$Sell_cf <-S_cf_quan
            b$Sell_cf_pps <- s$pps
            b$Sell_cf_basis <- S_cf_quan * s$pps
            

            # b$sell_quan <- s$quan    # cannot sell more than the buy_quan
            b$sell_quan <- b$quan
            b$sell_pps <- s$pps
            b$sell_basis <- b$quan * s$pps

            b$profit_loss <- b$sell_basis - b$buy_basis
            
            
            
            
            # b$sale <- sale

            s$type <- "SELL-CF"   
            
        }else if (s$quan == b$quan) {
            
            
             # b$sale_quan <- b$quan
            b$buy_quan <- s$quan
            b$buy_pps <-  b$pps
            b$buy_basis <-  b$buy_quan * b$pps


            b$sell_quan <- s$quan    # cannot sell more than the buy_quan
           # b$sell_quan <- b$quan
            b$sell_pps <- s$pps
            b$sell_basis <- s$quan * s$pps

            b$profit_loss <- b$sell_basis - b$buy_basis
            # b$sale <- sale

            s$type <- "SOLD"
            
            }
            
    
    

        return(b)   
}  # fct

updated_b <- update_b()
updated_b  
    

,index,date,type,co,quan,pps,basis,sale_quan,tally,cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,Sell_cf,Sell_cf_pps,Sell_cf_basis,TorF
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<int>,<int>,⋯,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<int>
1,1,2022-12-31,Buy,AMC,1050,4.07,4273.5,0,0,50,⋯,4.07,4070,1000,5.5124,5512.4,1442.4,0,0,0,0


In [11]:
# updated_b <- update_b()
# updated_b

### <font color = lime> Works with each sale type:( b > s , s > b, b = s, buys_cf, sells_cf )

In [12]:
evaluate_sale_type <- function () {
    
    cat(" This function displays the Sale Type based on the quan values from the updated_b data. ", "\n")
    cat(" Since the updated_b file is being used for evalutions, KEEP THINGS SIMPLE.", "\n\n")

    if(updated_b$quan > updated_b$sell_quan && updated_b$cf_quan > 0 ) {
        cat("This is a BUY carryforward transaction:  ( b_quan > s_quan ) ", "\n")
        cat("Since this is a BUY carryforward transaction, the carryforward values need to be appended to buys[1, ] position", "\n")
        cat(" T/F, good to go. Run:  snip buys[1, ] and sells[1,] ", "\n")
        cat(" And good to go. Record the updated_b into the permanent sales_record df ", "\n")
        cat(" And then begin a new loop " , "\n\n")
    } else if(updated_b$quan == updated_b$sell_quan && updated_b$cf_quan == 0 ) {
        cat("This is a simple transaction:  (b_quan == s_quan) ", "\n")
        cat(" T/F good to go.  Run snip buys[1,], sells[1,] ", "\n")
        cat(" And good to go. Record the updated_b into the permanent sales_record df ", "\n")
        cat(" And then begin a new loop ", "\n\n")            
    }else if (updated_b$quan < updated_b$sell_quan && updated_b$cf_quan == 0 ) {
        cat("This is a SELL carryforward transaction:  ( b_quan < s_quan ) ", "\n")
        cat("Since this is SELL carryforward transaction, the carryforward values need to be appended to sells[1, ] position", "\n")
        cat(" T/F, good to go. Run:  snip buys[1, ] and sells[1,] ", "\n")
        cat(" And good to go. Record the updated_b into the permanent sales_record df ", "\n")
        cat(" And then begin a new loop " , "\n\n")
        }
        
        cat("May need to examime more Sale Types ... when there is not enough Sell due to Sell CF values, then need to ..... ??? ", "\n\n")
    
    }  # fct

evaluate_sale_type()

 This function displays the Sale Type based on the quan values from the updated_b data.  
 Since the updated_b file is being used for evalutions, KEEP THINGS SIMPLE. 

This is a BUY carryforward transaction:  ( b_quan > s_quan )  
Since this is a BUY carryforward transaction, the carryforward values need to be appended to buys[1, ] position 
 T/F, good to go. Run:  snip buys[1, ] and sells[1,]  
 And good to go. Record the updated_b into the permanent sales_record df  
 And then begin a new loop  

May need to examime more Sale Types ... when there is not enough Sell due to Sell CF values, then need to ..... ???  



## <font color = yellow> Delete one row from buys and sells

In [13]:
save_sell_one <- data.frame()   #  keep for c/f >> Sell[1, ]

nrow(buys)

save_sell_one <- sells [1, ]
save_buy_one <- buys[-1,]

buys <- buys[-1, ]
sells <- sells[-1, ]

nrow(buys)

[1] 23

[1] 22

## <font color = yellow> Update the sales_records df w/ updated_b

In [14]:
sale_records <- rbind(sale_records, updated_b)
sale_records
                      

index,date,type,co,quan,pps,basis,sale_quan,tally,cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,Sell_cf,Sell_cf_pps,Sell_cf_basis,TorF
<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<int>,<int>,⋯,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<int>
1,2022-12-31,Buy,AMC,1050,4.07,4273.5,0,0,50,⋯,4.07,4070,1000,5.5124,5512.4,1442.4,0,0,0,0


## <font color = yellow> Bind  buys & sells - when buy/sell c/f exists


In [15]:
# good example from R:  ?add_row
df <- tibble(x = 1:3, y = 3:1)

df <- df %>% add_row(x = 4, y = 0, .before = 2)
df

x,y
<dbl>,<dbl>
1,3
4,0
2,2
3,1


In [16]:
if (sale_records$cf_quan[1] == 0 ) {
    cat("There is NO Buy c/f quan ", sale_records$cf_quan, "\n")
    }

In [17]:
# Here is is nec to update Buys[1, ]
if (sale_records$cf_quan[1] > 0 ) {
    cat("There IS Buy c/f quan ", sale_records$cf_quan, "\n")
    temp <- sale_records[1,]
    temp$type <- "Buy CF"
    temp$quan <- temp$cf_quan
    temp$pps <- temp$cf_pps
    temp$basis <- temp$cf_basis
    
    temp[ , 8:ncol(temp)] <- 0
    test <- buys
    test <- test %>% add_row(temp, .before = 1)
    buys <- test
    }

There IS Buy c/f quan  50 


In [18]:
if (sale_records$Sell_cf[1] == 0 ) {
    cat("There is NO Sell c/f ", sale_records$Sell_quan, "\n")
    }

There is NO Sell c/f  


In [19]:
# Here it is nec to update the Sells [1, ] 
if (sale_records$Sell_cf[1] > 0 ) {
    cat("There IS Sell c/f ", sale_records$Sell_cf, "\n")
    
    temp <- save_sell_one
    temp2 <-  temp <- sale_records[1,]
    temp[ , 8:ncol(save_sell_one)] <- 0
   
    
    temp$type <- "Sell CF"
    temp$quan <- temp2$Sell_cf
    temp$pps <- temp2$Sell_cf_pps
    temp$basis <- temp2$Sell_cf_basis

    sell_one <- temp
    
    test <- sells
    test <- test %>% add_row(sell_one, .before = 1)
    sells <- test
    
    
    
    
    
    }

In [20]:
#  Here, there are no c/f issues , t/f , return()
if (sale_records$cf_quan[1] == 0 && sale_records$Sell_cf[1] == 0 ) {
    cat("There is NO Buy c/f or Sell c/f ", sale_records$cf_quan, " \t", sale_records$Sell_cf, "\n")
    }


In [21]:
nrow(buys)
head(buys,3)

[1] 23

,index,date,type,co,quan,pps,basis,sale_quan,tally,cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,Sell_cf,Sell_cf_pps,Sell_cf_basis,TorF
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,2022-12-31,Buy CF,AMC,50,4.070,203.5,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
2,3,2023-01-20,Buy,AMC,1000,5.625,5625.0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
3,5,2023-01-23,Buy,AMC,1000,5.788,5788.0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


In [22]:
nrow(sells)
head(sells, 3)

[1] 17

,index,date,type,co,quan,pps,basis,sale_quan,tally,cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,Sell_cf,Sell_cf_pps,Sell_cf_basis,TorF
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<int>,<int>,⋯,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<int>
4,4,2023-02-03,Sell,AMC,1000,6.68255,6682.55,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
6,6,2023-02-03,Sell,AMC,1000,6.25500,6255.00,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
8,8,2023-02-03,Sell,AMC,1000,6.19010,6190.10,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


In [23]:
#  # Only once per loop
# test <- test %>% add_row(temp, .before = 1)
# head(test, 3)

In [24]:
sale_records

index,date,type,co,quan,pps,basis,sale_quan,tally,cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,Sell_cf,Sell_cf_pps,Sell_cf_basis,TorF
<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<int>,<int>,⋯,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<int>
1,2022-12-31,Buy,AMC,1050,4.07,4273.5,0,0,50,⋯,4.07,4070,1000,5.5124,5512.4,1442.4,0,0,0,0


### <font color = lime> Got it! 

In [25]:
print(t(b))

              1           
index         "1"         
date          "2022-12-31"
type          "Buy"       
co            "AMC"       
quan          "1050"      
pps           "4.07"      
basis         "4273.5"    
sale_quan     "0"         
tally         "0"         
cf_quan       "0"         
cf_pps        "0"         
cf_basis      "0"         
buy_quan      "0"         
buy_pps       "0"         
buy_basis     "0"         
sell_quan     "0"         
sell_pps      "0"         
sell_basis    "0"         
profit_loss   "0"         
Sell_cf       "0"         
Sell_cf_pps   "0"         
Sell_cf_basis "0"         
TorF          "0"         


In [26]:
stopps #  Old code

ERROR: Error in eval(expr, envir, enclos): object 'stopps' not found


## <font color = yellow> Continue! Inputs: buys, sells, sale_records 

In [27]:
nrow(buys)    # if  b > s, then -1, +1 , t/f 23/17
nrow(sells)   # if  s > b, then -1, +1 , t/f 22/18
nrow(sale_records)

# 

[1] 23

[1] 17

[1] 1

In [28]:
b <- buys[1,]
s <- sells[1,]
b
s

,index,date,type,co,quan,pps,basis,sale_quan,tally,cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,Sell_cf,Sell_cf_pps,Sell_cf_basis,TorF
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,2022-12-31,Buy CF,AMC,50,4.07,203.5,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


,index,date,type,co,quan,pps,basis,sale_quan,tally,cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,Sell_cf,Sell_cf_pps,Sell_cf_basis,TorF
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<int>,<int>,⋯,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<int>
4,4,2023-02-03,Sell,AMC,1000,6.68255,6682.55,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


In [29]:
print(t(b))

              1           
index         "1"         
date          "2022-12-31"
type          "Buy CF"    
co            "AMC"       
quan          "50"        
pps           "4.07"      
basis         "203.5"     
sale_quan     "0"         
tally         "0"         
cf_quan       "0"         
cf_pps        "0"         
cf_basis      "0"         
buy_quan      "0"         
buy_pps       "0"         
buy_basis     "0"         
sell_quan     "0"         
sell_pps      "0"         
sell_basis    "0"         
profit_loss   "0"         
Sell_cf       "0"         
Sell_cf_pps   "0"         
Sell_cf_basis "0"         
TorF          "0"         


In [30]:
sale_records


index,date,type,co,quan,pps,basis,sale_quan,tally,cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,Sell_cf,Sell_cf_pps,Sell_cf_basis,TorF
<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<int>,<int>,⋯,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<int>
1,2022-12-31,Buy,AMC,1050,4.07,4273.5,0,0,50,⋯,4.07,4070,1000,5.5124,5512.4,1442.4,0,0,0,0


## <font color = yellow> Update b

In [31]:
# update b based on Sale Type: 

update_b <- function(b) {
    
     cat("Inside update_b, test for: $quan < s$quan && b$type == 'Buy CF': ", "\n")
    
        if(b$quan < s$quan && b$type == 'Buy CF') {
            cat("True", "\n")
            }

        sale <- s$quan
        bs_ratio <- b$quan / s$quan
        cf_quan <-  b$quan - s$quan

        # assign the cf values - if they exist
         #  b > s ... restricted to Buy only
        if ( b$quan > s$quan && b$type == "Buy") {  
            
            # b$sale_quan <- b$quan
            b$buy_quan <- s$quan
            b$buy_pps <-  b$pps
            b$buy_basis <-  b$buy_quan * b$pps


            b$sell_quan <- s$quan    # cannot sell more than the buy_quan
           # b$sell_quan <- b$quan
            b$sell_pps <- s$pps
            b$sell_basis <- s$quan * s$pps

            b$cf_quan <- (b$quan - s$quan)        
            b$cf_pps <- b$pps
            b$cf_basis <- cf_quan * b$pps           

            b$profit_loss <- b$sell_basis - b$buy_basis
            # b$sale <- sale

            s$type <- "Buy-CF"
            
            
            
            
            
            
            
            
            
            
            
            
            
            
            
            
            
            
            
            
            
            }else if( b$quan < s$quan && b$type == 'Buy CF') {    
                 
        cat("Inside update_b, and the 5'th else if ... when there exists 'Buy-CF' ", "\n")
            
            temp <- b   # protect b from errors
            
            
        # Sale 
            
            
            temp$type <- "Buy-2xCF"
           
            
            temp$sell_quan <- temp$quan
            temp$sell_pps <- sale_records$sell_pps[1]
            temp$sell_basis <- temp$sell_quan * sale_records$sell_pps
            temp$profit_loss <- temp$sell_basis - temp$basis
           
        # Sell c/f 

#              temp$cf_quan <- (b$quan - s$quan)        
#              temp$cf_pps <- b$pps
#              temp$cf_basis <- cf_quan * b$pps             
            
            

            
        # 
            
            # update Sale c/f  from sale_records
            
            
            temp$Sell_cf  <-  sale_records$sell_quan - sale_records$cf_quan 
            temp$Sell_cf_pps   <- sale_records$sell_pps
            
         #  temp$Sell_cf_basis <- temp$Sell_cf * temp$Sell_cf_pps
            
            # update b with temp
            b <- temp

            s$type <- "Buy-2xCF"       
    
            
            
            
            
            
            
            
            
            
            
            
            
            
            
            
            
            
            
            
            
            
            
            
            
            
            
       # ( s > b ), has sell carryforward
        }else if(s$quan > b$quan) {
            
            
            b$sale_quan <- b$quan
           # b$buy_quan <- s$quan 
            b$buy_quan <- b$quan
            b$buy_pps <-  b$pps
            b$buy_basis <-  b$buy_quan * b$pps

        
            # Sell c/f
            S_cf_quan <- s$quan - b$quan           
            b$Sell_cf <-S_cf_quan
            b$Sell_cf_pps <- s$pps
            b$Sell_cf_basis <- S_cf_quan * s$pps
            

            # b$sell_quan <- s$quan    # cannot sell more than the buy_quan
            b$sell_quan <- b$quan
            b$sell_pps <- s$pps
            b$sell_basis <- b$quan * s$pps

            b$profit_loss <- b$sell_basis - b$buy_basis
            
            
            
            
            # b$sale <- sale

            s$type <- "SELL-CF"   
            
        }else if (s$quan == b$quan) {
            
        
            
            
             # b$sale_quan <- b$quan
            b$buy_quan <- s$quan
            b$buy_pps <-  b$pps
            b$buy_basis <-  b$buy_quan * b$pps


            b$sell_quan <- s$quan    # cannot sell more than the buy_quan
           # b$sell_quan <- b$quan
            b$sell_pps <- s$pps
            b$sell_basis <- s$quan * s$pps

            b$profit_loss <- b$sell_basis - b$buy_basis
            # b$sale <- sale

            s$type <- "SOLD"
            
            }
        
   
#         }else if( b$quan < s$quan && b$type == 'Buy CF') {    
                 
#         cat("Inside update_b, and the 5'th else if ... when there exists 'Buy-CF' ", "\n")
            
#             temp <- b   # protect b from errors
            
            
#         # Sale 
#             temp$type <- "Buy-2xCF"
#             temp$sell_quan <- temp$quan
#             temp$sell_pps <- sale_records$sell_pps[1]
#             temp$sell_basis <- temp$sell_quan * temp_sell_pps

           
#         # Sell c/f 

#              temp$cf_quan <- (b$quan - s$quan)        
#              temp$cf_pps <- b$pps
#              temp$cf_basis <- cf_quan * b$pps           

#             temp$profit_loss <- b$sell_basis - b$buy_basis
#         # 
            
#             # update b from temp
            
#             b <- temp

#             s$type <- "Buy-2CF"
            
#         }
      

        return(b)   
}  # fct

updated_b <- update_b(b)
updated_b  
    

Inside update_b, test for: $quan < s$quan && b$type == 'Buy CF':  
True 
Inside update_b, and the 5'th else if ... when there exists 'Buy-CF'  


,index,date,type,co,quan,pps,basis,sale_quan,tally,cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,Sell_cf,Sell_cf_pps,Sell_cf_basis,TorF
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>
1,1,2022-12-31,Buy-2xCF,AMC,50,4.07,203.5,0,0,0,⋯,0,0,50,5.5124,275.62,72.12,950,5.5124,0,0


In [32]:
print(t(updated_b))

              1           
index         "1"         
date          "2022-12-31"
type          "Buy-2xCF"  
co            "AMC"       
quan          "50"        
pps           "4.07"      
basis         "203.5"     
sale_quan     "0"         
tally         "0"         
cf_quan       "0"         
cf_pps        "0"         
cf_basis      "0"         
buy_quan      "0"         
buy_pps       "0"         
buy_basis     "0"         
sell_quan     "50"        
sell_pps      "5.5124"    
sell_basis    "275.62"    
profit_loss   "72.12"     
Sell_cf       "950"       
Sell_cf_pps   "5.5124"    
Sell_cf_basis "0"         
TorF          "0"         


## <font color = yellow> Delete one row from buys and sells
    
#### Nec to start loop at the first (unprocessed rows in buys / sells at the top of the loop

In [33]:
save_sell_one <- data.frame()   #  keep for c/f >> Sell[1, ]

nrow(buys)

save_sell_one <- sells [1, ]
save_buy_one <- buys[-1,]

buys <- buys[-1, ]
sells <- sells[-1, ]

nrow(buys)

[1] 23

[1] 22

## <font color = yellow> Update the sales_records df w/ updated_b

In [34]:
sale_records <- rbind(sale_records, updated_b)
sale_records

index,date,type,co,quan,pps,basis,sale_quan,tally,cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,Sell_cf,Sell_cf_pps,Sell_cf_basis,TorF
<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>
1,2022-12-31,Buy,AMC,1050,4.07,4273.5,0,0,50,⋯,4.07,4070,1000,5.5124,5512.40,1442.40,0,0.0000,0,0
1,2022-12-31,Buy-2xCF,AMC,50,4.07,203.5,0,0,0,⋯,0.00,0,50,5.5124,275.62,72.12,950,5.5124,0,0


## <font color = yellow> 